In [1]:
from tkinter import Tk,Canvas
import random
import queue

canvas_height = 450
canvas_width = 750
turn = ["U","L","D","R"]
turn_i = random.randint(0,3)
robot_i = 0
previous_nodes = []
x0 = 0
y0 = 0
turn0 = [0,1,2,3]
turn0_i = 0
visited_nodes = []
backtrack_nodes = []
obstacles = []
b = 0

def get_maze(maze):
    with open(maze) as f:
        lines = f.read().splitlines()
        return [line.strip() for line in lines]

def get_robot_xy():
    for y, row in enumerate(maze):
        x = row.find("R")
        if(x != -1):
            return x, y

def draw_maze():
    for y, row in enumerate(maze):
        for x, clm in enumerate(row):
            if(is_wall(x, y)):
                draw_wall(x, y)
    


def draw_wall(x, y):
    draw(x, y, "black")

def draw_robot(x, y):
    draw(x, y, "orange")

def draw_footprint(x, y):
    draw(x, y, "grey")

def draw(x,y,color):

    maze_height = len(maze)
    maze_width = len(maze[1])
    height = canvas_height/maze_height
    width = canvas_width/maze_width

    x1, y1 = (x*width, y*height)
    x2, y2 = (x1+width, y1+height)
    w.create_rectangle(x1, y1, x2, y2, fill=color, outline=color)

def is_wall(x, y):
    return maze[y][x] == '#'

def is_exit(x, y):
    return maze[y][x] == 'E'

def trystep():
    global x,y,turn,turn_i,x0,y0
    if turn[turn_i] == "U":
        if maze[y+1][x] == "#":
            return False
        draw_footprint(x,y)
        y = y + 1
        draw_robot(x,y)
        return True
    elif turn[turn_i] == "L":
        if maze[y][x-1] == "#":
            return False
        draw_footprint(x,y)
        x = x - 1
        draw_robot(x,y)
        return True
    elif turn[turn_i] == "D":
        if maze[y-1][x] == "#":
            return False
        draw_footprint(x,y)
        y = y - 1
        draw_robot(x,y)
        return True
    elif turn[turn_i] == "R":
        if maze[y][x+1] == "#":
            return False
        draw_footprint(x,y)
        x = x + 1
        draw_robot(x,y)
        return True

def fulltrystep():
    global turn0_i,x0,y0

    if trystep():
        if turn0_i == 0:
            y0 = y0 + 1
        elif turn0_i == 1:
            x0 = x0 - 1
        elif turn0_i == 2:
            y0 = y0 - 1
        elif turn0_i == 3:
            x0 = x0 + 1
    else:
        return False



def turn_left():
    global turn_i,turn0_i
    if turn_i == 3:
        turn_i = 0
    else:
        turn_i = turn_i + 1

    if turn0_i == 3:
        turn0_i = 0
    else:
        turn0_i = turn0_i + 1
    

def reset():     
    global turn0_i   
    if turn0_i == 1:
        turn_left()
        turn_left()
        turn_left()
        turn0_i = 0
    elif turn0_i == 2:
        turn_left()
        turn_left()
        turn0_i = 0
    elif turn0_i == 3:
        turn_left()

def backtrack():
    reset()
    
    bx, by = visited_nodes[len(visited_nodes) - 1 - b]
    gx = bx - x
    gy = by - y
    while gx != 0 and gy != 0:
        if gx <= -1:
            turn_left()
            trystep()
            continue
        
        if gx >= 1:
            turn_left()
            turn_left()
            turn_left()
            trystep()
            continue

        if gy >= 1:
            trystep()
            continue
        if gy <= -1:
            turn_left()
            turn_left()
            trystep()



def move():
    global x0,y0,visited_nodes
    reset()
    visi = True
    for n in visited_nodes:
        if n == (x0,y0+1):
            visi = False
            break
    if visi:
        if fulltrystep():
            visited_nodes.append((x0,y0))
            b = 0
            return
        else:
            obstacles.append((x0,y0+1))
    turn_left()
    visi = True
    for n in visited_nodes:
        if n == (x0-1,y0):
            visi = False
            break
    if visi:
        if trystep():
            visited_nodes.append((x0,y0))
            b = 0
            return
        else:
            obstacles.append((x0-1,y0))
    turn_left()
    visi = True
    for n in visited_nodes:
        if n == (x0,y0-1):
            visi = False
            break
    if visi:
        if fulltrystep():
            visited_nodes.append((x0,y0))
            b = 0
            return
        else:
            obstacles.append((x0,y0-1))
    turn_left()
    visi = True
    for n in visited_nodes:
        if n == (x0+1,y0):
            visi = False
            break
    if visi:
        if fulltrystep():
            visited_nodes.append((x0,y0))
            b = 0
            return
        else:
            obstacles.append((x0+1,y0))

    backtrack()


def solve():
    move()

    root.after(50, solve)

def main():
    global root,w,maze,x,y
    maze = get_maze("mymaze.txt")
    x,y = get_robot_xy()
    root = Tk()
    root.title("Map")
    w = Canvas(root, bg="white", width=canvas_width, height=canvas_height)
    w.pack()
    draw_maze()
    solve()
    root.mainloop()



main()
